--------
--------
# *Title, Description/Problem Statement of Project*
--------
--------

# PART 1 - Data Extraction  

## Import neccessary libraries

In [1]:
import numpy as np
import pandas as pd
import lxml
import lxml.etree

## Building Dataframe from webpage

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)
df = data[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Filtering Dataframe to remove 'Not Assigned' Boroughs

In [3]:
filter_df = df.loc[ df['Borough'] != 'Not assigned']
print(f"Not assigned Boroughs: {(filter_df.Borough == 'Not assigned').sum()}")

Not assigned Boroughs: 0


In [4]:
print(f"Not assigned Neighbourhoods: {(filter_df.Neighbourhood == 'Not assigned').sum()}")

Not assigned Neighbourhoods: 0


In [5]:
filter_df.reset_index(drop=True, inplace=True)
filter_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
print(f"Shape of filtered Dataframe: {filter_df.shape}")

Shape of filtered Dataframe: (103, 3)


--------
--------
# Save *filter_df* as pickle file - *venues.pkl*
# PART 2 to PART 3.2 in file - *selectBoroughs.py*
--------
--------

# PART 2 - Geospatial data (Latitudes & Longitudes) Extraction

## Import neccessary libraries

In [7]:
#!pip install geocoder --quiet
import geocoder
from geopy.geocoders import Nominatim

## Using Postal codes for Latitudes & Longitudes

In [8]:
#define empty lists for geospatial values and None values
lat = []
long = []
nones = []

#initialize geolocator
geolocator = Nominatim(user_agent = 'toronto_zipcodes_M')

#loop through filter_df['Postal Code'] values and append to the respective lists
for code in filter_df['Postal Code']:
    location = geolocator.geocode(code)
    if location is None:
        nones.append(None)
        nones.append(None)
    else:
        latitude = location.latitude
        longitude = location.longitude
    lat.append(latitude)
    long.append(longitude)


In [9]:
print(f"Total non-None values: {len(lat)}, {len(long)}")
print(f"Total None values: {len(nones)}")

Total non-None values: 103, 103
Total None values: 118


## Adding Latitudes & Longitudes values in filter_df

In [10]:
filter_df['Latitude'] = lat
filter_df['Longitude'] = long
filter_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,-12.198334,-76.962310
1,M4A,North York,Victoria Village,49.484290,8.467000
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",45.440588,28.018025
3,M6A,North York,"Lawrence Manor, Lawrence Heights",53.794164,-1.752006
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",44.428198,26.165951


# PART-3 Explore & Cluster Neighbourhoods

## PART-3.1 Explore Neighbourhoods in Boroughs ending with 'Toronto'

### Import necessary libraries

In [11]:
import re
import folium
import requests

### Apply *regex* function to extract Borough names ending with 'Toronto'

In [12]:
string = filter_df['Borough'].values
regex = r"\w*\sToronto"

borough = []
none_vals = []
for row in string:
    val = re.findall(regex, row)
    if val == []:
        none_vals.append(val)
    else:
        borough.append(val)

In [13]:
chosen_boroughs = np.unique(np.array(borough))
print('Boroughs chosen for exploring venues:\n',chosen_boroughs)

Boroughs chosen for exploring venues:
 ['Central Toronto' 'Downtown Toronto' 'East Toronto' 'West Toronto']


### Subset Dataframe with chosen Boroughs for exploring

In [51]:
explore_df = filter_df.loc[filter_df['Borough'].isin(chosen_boroughs)].reset_index(drop=True)

#change of spelling from 'Neighbourhood' to 'Neighborhood'
cols = {'Postal Code':'Postal Code',  'Borough':'Borough', 'Neighbourhood':'Neighborhood', 'Latitude':'Latitude', 'Longitude':'Longitude'}
explore_df.rename(columns = cols, inplace=True)
explore_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",45.440588,28.018025
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",44.428198,26.165951
2,M5B,Downtown Toronto,"Garden District, Ryerson",45.440820,28.016137
3,M5C,Downtown Toronto,St. James Town,-6.173246,39.207869
4,M4E,East Toronto,The Beaches,43.644110,-79.588907


In [15]:
print(f"Total unique Boroughs: {explore_df.Borough.nunique()}\nTotal unique Neighborhoods: {explore_df.Neighborhood.nunique()}")

Total unique Boroughs: 4
Total unique Neighborhoods: 39


### Map chosen Boroughs and their Neighbourhoods in Toronto 

In [16]:
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent='Toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f"Coordinates of Toronto are: {latitude}, {longitude}")

Coordinates of Toronto are: 43.6534817, -79.3839347


In [17]:
toronto_map = folium.Map(location= [latitude, longitude], zoom_start=10)

#add Neighborhood markers on toronto_map
for lat, long, bor, hood in zip(explore_df['Latitude'], explore_df['Longitude'],explore_df['Borough'],explore_df['Neighborhood']):
    label = folium.Popup('{}, {}'.format(bor, hood), parse_html=True)
    folium.CircleMarker([lat, long],
        radius=4,
        popup=label,
        color='cyan',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(toronto_map)

toronto_map

---------
---------
# API details in file - *secret.py*
# Function (getNearbyVenues) definition in file - *nearbyVenues.py*
# Extract Venues, Encode & Group in file - *hoodVenues.py*
# Export *toronto_grouped* dataframe as pickle file - *venueCat_grouped.pkl*
----------
----------

## PART-3.2 Build & Analyze nearby venues in Neighbourhoods

### Build nearby venues dataframe

In [1]:
#Credentials
CLIENT_ID = 'CXC1D1CNWMCS54XHC3M0VLPRLBCPQQMID0OZC04Z0VYTMSAU' 
CLIENT_SECRET = 'OQRFM1BNLVMREJ3N3VJBAWGKU2ERVDEBC3Q1M2UXHBVNDBN3' 
VERSION = '20201201' 
LIMIT = 100

SyntaxError: invalid syntax (<ipython-input-1-649fb978b4b6>, line 3)

### Using defined-function from *Neighborhoods-New-York* Lab for exploring Neighborhoods in chosen Boroughs
#### * increased radius from 500 to 1000 of function getNearbyVenues, hence the number of Venues could be more

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(explore_df.Neighborhood, explore_df.Latitude, explore_df.Longitude)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [21]:
print(f"Total Rows:{toronto_venues.shape[0]}, Total Columns:{toronto_venues.shape[1]}")
toronto_venues.head()

Total Rows:1875, Total Columns:7


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",45.440588,28.018025,LIDL,45.439468,28.029159,Supermarket
1,"Regent Park, Harbourfront",45.440588,28.018025,Centru,45.442525,28.017562,Plaza
2,"Regent Park, Harbourfront",45.440588,28.018025,Restaurant HAZARD,45.440675,28.028740,Mediterranean Restaurant
3,"Regent Park, Harbourfront",45.440588,28.018025,Golden Chicken,45.433389,28.025602,Fast Food Restaurant
4,"Queen's Park, Ontario Provincial Government",44.428198,26.165951,Mega Image,44.425392,26.165526,Supermarket


In [22]:
print(f"Total unique Venue categories: {toronto_venues['Venue Category'].nunique()}")

Total unique Venue categories: 231


#### Important Assumption for Analysis:

* Based on .nunique method, there are _'N'_ unique venue categories
* BUT, one of the Venue Category is _'Neighborhood'_. 
* THIS makes .groupby('Neighborhood') after onehot-encoding error-prone ---> _(ValueError: Grouper for 'Neighborhood' not 1-dimensional)_
* Hence, excluding _'Neighborhood'_ from Venue categories
* ASSUMED Total unique Venue categories = _'N-1'_

In [23]:
print("Venues with 'Neighborhood' Venue Category:\n")
toronto_venues.loc[toronto_venues['Venue Category'] == 'Neighborhood']


Venues with 'Neighborhood' Venue Category:



,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
61,"Garden District, Ryerson",45.440820,28.016137,Piața Energiei - General,45.440053,28.021103,Neighborhood
88,Berczy Park,43.642106,-79.377445,Harbourfront,43.639526,-79.380688,Neighborhood
219,"Richmond, Adelaide, King",43.649901,-79.383718,Downtown Toronto,43.653232,-79.385296,Neighborhood
319,"Dufferin, Dovercourt Village",43.649901,-79.383718,Downtown Toronto,43.653232,-79.385296,Neighborhood
409,"Harbourfront East, Union Station, Toronto Islands",43.639259,-79.382840,Harbourfront,43.639526,-79.380688,Neighborhood


### Analyze nearby venues dataframe

#### one-hot encode Venue Category for statistics by Neighbourhood

In [24]:
encode_df = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')
encode_df.head()

,Airport,Airport Food Court,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# drop the Venue category 'Neighborhood'
encode_df.drop('Neighborhood', axis=1, inplace=True)

In [26]:
# add 'Neighborhood' columns with Neighborhood names 
encode_df['Neighborhood'] = toronto_venues[['Neighborhood']]

# move Neighborhood column to the '0' position
fixed_columns = ['Neighborhood'] + list(encode_df.columns[encode_df.columns!='Neighborhood'])
encode_df = encode_df[fixed_columns]
encode_df.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Queen's Park, Ontario Provincial Government",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
encode_df.shape

(1875, 231)

In [28]:
toronto_grouped = encode_df.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,...,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Xinjiang Restaurant,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.010000,0.000000,0.02,0.00,0.020000,0.00,...,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.017241,0.000000,0.00,0.00,0.017241,0.00,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.017544,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.000000,0.017544,0.00,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.00,0.010000,0.00,...,0.00,0.010000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.030000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.071429,0.000000,0.00,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.071429,0.000000,0.00,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.017544,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.000000,0.017544,0.00,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


---------
---------
## *Import venueCat_grouped.pkl file - venueCat_grouped dataframe*
## *Automate and define Function for commonly visited venues in file - popularVenues.py*
---------
---------

### Extract Top 10 venues from each Neighborhood (_using code from Neighborhoods-New-York Lab_)

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_top_venues = pd.DataFrame(columns=columns)
neighborhoods_top_venues['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_top_venues.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_top_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Park,Beer Bar,Restaurant,Japanese Restaurant,Concert Hall,Art Gallery,Aquarium
1,"Brockton, Parkdale Village, Exhibition Place",Bakery,Breakfast Spot,Indian Restaurant,Coffee Shop,Athletics & Sports,Café,Park,Nightclub,Tibetan Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Gay Bar,Park,Diner,Coffee Shop,Pub,Café,Thai Restaurant,Gastropub,Restaurant,Japanese Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Hotel,Yoga Studio,Beer Bar,Café,Restaurant,French Restaurant,Aquarium,Pizza Place
4,Central Bay Street,Fast Food Restaurant,Sandwich Place,Hotel,Home Service,Moving Target,Video Store,Construction & Landscaping,Bar,Liquor Store,Intersection


----------
----------
## Import *popularVenues.py* & collect *topVenues dataframe*
## Cluster & Map *topVenues*
----------
----------

## PART-3.3 Clustering Neighborhoods by neighborhoods_top_venues dataframe 

### Import necessary libraries

In [62]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Clustering _toronto_grouped_ dataframe

In [117]:
k = 4

#Drop 'Neighborhood' column for clustering
toronto_cluster = toronto_grouped.drop('Neighborhood', axis=1)

#KMeans clustering
km_cluster = KMeans(n_clusters=k, random_state=90)
km_cluster.fit(toronto_cluster)
labels = km_cluster.labels_
print(f"Cluster labels of toronto_grouped dataframe:\n{labels}")


Cluster labels of toronto_grouped dataframe:
[0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 1
 1 0]


### Mapping clusters of _toronto_venues_cluster_

In [119]:
#inserting Cluster labels
neighborhoods_top_venues.insert(0, 'Cluster Label', labels)

#joining cluster data with origin data
toronto_venues_cluster = explore_df.join(neighborhoods_top_venues.set_index('Neighborhood'), on='Neighborhood')
toronto_venues_cluster.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",45.440588,28.018025,0,Supermarket,Plaza,Fast Food Restaurant,Mediterranean Restaurant,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",44.428198,26.165951,0,Restaurant,Supermarket,Bus Station,Park,Pizza Place,Pharmacy,Gym / Fitness Center,Metro Station,Electronics Store,Lounge
2,M5B,Downtown Toronto,"Garden District, Ryerson",45.440820,28.016137,0,Bar,Bowling Alley,Bridal Shop,Plaza,Food,Mediterranean Restaurant,Auto Dealership,Falafel Restaurant,Food Court,Food & Drink Shop
3,M5C,Downtown Toronto,St. James Town,-6.173246,39.207869,3,Grocery Store,Heliport,Yoga Studio,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,M4E,East Toronto,The Beaches,43.644110,-79.588907,0,Baseball Field,Golf Course,Racetrack,Flower Shop,College Rec Center,Garden,Gas Station,Park,Yoga Studio,Flea Market


In [122]:
print(f"Neighborhoods per Cluster:\n{toronto_venues_cluster.groupby('Cluster Label')['Neighborhood'].count()}")

Neighborhoods per Cluster:
Cluster Label
0    34
1     2
2     2
3     1
Name: Neighborhood, dtype: int64


In [121]:
toronto_cluster_map = folium.Map(location= [latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add Neighborhood markers on toronto_map
for lat, long, hood, cluster in zip(toronto_venues_cluster['Latitude'], toronto_venues_cluster['Longitude'],toronto_venues_cluster['Neighborhood'], toronto_venues_cluster['Cluster Label'] ):
    label = folium.Popup(f"{str(hood)} + Cluster:{str(cluster)}", parse_html=True)
    folium.CircleMarker([lat, long],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.9,
        parse_html=False).add_to(toronto_cluster_map)

toronto_cluster_map